In [ ]:
import pandas as pd
import glob
import os

path = r'C:\Users\user\Downloads\NYC_2024'

print(f"Searching for CSV (.csv) files in: {path}")
all_files = glob.glob(os.path.join(path, "*.csv")) 

if not all_files:
    print("\n--- ERROR ---")
    print("No CSV (.csv) files were found in the specified directory.")
    print("Please double-check that your CSV files are in the NYC_2024 folder.")
else:
    print(f"Found {len(all_files)} files. Combining now...")
    li = []
    for filename in all_files:
        df_chunk = pd.read_csv(filename, index_col=None, header=0, low_memory=False)
        li.append(df_chunk)

    df = pd.concat(li, axis=0, ignore_index=True)
    print("Files combined successfully!")
    print("\n--- Initial Data Diagnosis ---")
    
    df.info()
    print("\n--- Missing Values Count ---")
    print(df.isnull().sum())

Searching for CSV (.csv) files in: C:\Users\user\Downloads\NYC_2024
Found 6 files. Combining now...
Files combined successfully!

--- Initial Data Diagnosis ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750801 entries, 0 to 750800
Data columns (total 42 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unique Key                      750801 non-null  int64  
 1   Created Date                    750801 non-null  object 
 2   Closed Date                     741825 non-null  object 
 3   Agency                          750801 non-null  object 
 4   Agency Name                     750801 non-null  object 
 5   Complaint Type                  750801 non-null  object 
 6   Descriptor                      734341 non-null  object 
 7   Location Type                   661871 non-null  object 
 8   Incident Zip                    742500 non-null  object 
 9   Incident Address                722257 no

In [ ]:
df['Created Date'] = pd.to_datetime(df['Created Date'], errors='coerce')
df['Closed Date'] = pd.to_datetime(df['Closed Date'], errors='coerce')

C:\Users\user\AppData\Local\Temp\ipykernel_14668\3042999796.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Created Date'] = pd.to_datetime(df['Created Date'], errors='coerce')


In [ ]:
print("Dropping unnecessary columns...")
columns_to_drop = [
    'Unnamed: 41', 'Cross Street 1', 'Cross Street 2', 
    'Intersection Street 1', 'Intersection Street 2', 'BBL',
    'X Coordinate (State Plane)', 'Y Coordinate (State Plane)', 'Latitude', 
    'Longitude', 'Location', 'Vehicle Type', 'Taxi Company Borough', 
    'Taxi Pick Up Location', 'Bridge Highway Name', 'Bridge Highway Direction', 
    'Road Ramp', 'Bridge Highway Segment'
]
df.drop(columns=columns_to_drop, inplace=True)
print("Columns dropped successfully.")

print("\nCreating 'Resolution Time' column...")
df['Resolution Time'] = df['Closed Date'] - df['Created Date']
print("'Resolution Time' created successfully.")

print("\n--- First 5 Rows with Resolution Time ---")
print(df[['Created Date', 'Closed Date', 'Resolution Time']].head())

print("\n--- Updated Data Info ---")
df.info()

Dropping unnecessary columns...
Columns dropped successfully.

Creating 'Resolution Time' column...
'Resolution Time' created successfully.

--- First 5 Rows with Resolution Time ---
         Created Date         Closed Date Resolution Time
0 2024-01-15 23:59:59 2024-01-17 09:23:12 1 days 09:23:13
1 2024-01-15 23:59:45 2024-01-17 18:41:04 1 days 18:41:19
2 2024-01-15 23:59:44 2024-01-17 14:16:30 1 days 14:16:46
3 2024-01-15 23:59:39 2024-01-17 14:17:06 1 days 14:17:27
4 2024-01-15 23:59:38 2024-01-16 00:39:45 0 days 00:40:07

--- Updated Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750801 entries, 0 to 750800
Data columns (total 25 columns):
 #   Column                          Non-Null Count   Dtype          
---  ------                          --------------   -----          
 0   Unique Key                      750801 non-null  int64          
 1   Created Date                    750800 non-null  datetime64[ns] 
 2   Closed Date                     438081 non-nul

In [ ]:
top_10_complaints = df['Complaint Type'].value_counts().head(10)
print("--- Top 10 Complaint Types ---")
print(top_10_complaints)

--- Top 10 Complaint Types ---
Complaint Type
Illegal Parking            116050
HEAT/HOT WATER             100799
Noise - Residential         64785
Blocked Driveway            40667
UNSANITARY CONDITION        26751
Street Condition            17921
Abandoned Vehicle           16627
PAINT/PLASTER               16448
PLUMBING                    16296
Noise - Street/Sidewalk     15085
Name: count, dtype: int64


In [ ]:
top_10_agencies = df['Agency Name'].value_counts().head(10)
print("\n--- Top 10 Busiest Agencies ---")
print(top_10_agencies)


--- Top 10 Busiest Agencies ---
Agency Name
New York City Police Department                       299521
Department of Housing Preservation and Development    216015
Department of Sanitation                               61944
Department of Transportation                           50451
Department of Environmental Protection                 36008
Department of Buildings                                23613
Department of Parks and Recreation                     18917
Department of Health and Mental Hygiene                18042
Taxi and Limousine Commission                           8345
Department of Homeless Services                         6808
Name: count, dtype: int64


In [ ]:
average_resolution_time = df['Resolution Time'].mean()
print(f"\n--- Average Resolution Time ---")
print(average_resolution_time)


--- Average Resolution Time ---
12 days 12:07:26.374066381


In [ ]:
top_5_boroughs = df['Borough'].value_counts().head(5)
print("\n--- Top 5 Boroughs by Complaint Volume ---")
print(top_5_boroughs)


--- Top 5 Boroughs by Complaint Volume ---
Borough
BROOKLYN         233354
QUEENS           180017
MANHATTAN        156645
BRONX            154244
STATEN ISLAND     25962
Name: count, dtype: int64


In [ ]:
df.to_csv('NYC_311_Q1_2024_Cleaned.csv', index=False)

print("\nCleaned data has been saved to NYC_311_Q1_2024_Cleaned.csv")


Cleaned data has been saved to NYC_311_Q1_2024_Cleaned.csv


In [ ]:
import sqlite3

db_file_name = 'NYC_Data.db'

table_name = 'Complaints_2024_Q1'

print(f"Loading data into SQL database: {db_file_name}...")

conn = sqlite3.connect(db_file_name)

df.to_sql(table_name, conn, if_exists='replace', index=False)

conn.close()

print("Data successfully loaded into SQL database.")

Loading data into SQL database: NYC_Data.db...


C:\Users\user\AppData\Local\Temp\ipykernel_14668\3973480551.py:18: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  df.to_sql(table_name, conn, if_exists='replace', index=False)


Data successfully loaded into SQL database.
